In [1]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [31]:
from Scanner import Scanner

class Parser :
    def __init__(self, source) :
        self.source = source
        self.scanner = Scanner(source)
        self.start = self.Stmt
        self.currentToken = None
        self.currentValue = None
    
    def error(self,message) :
        print("Error: " + message)
       
    def emit(self,tag,item) :
        print([tag,item])
        
    def scan(self) :
        self.currentToken, self.currentValue = self.scanner.nextToken()
        
    def expect(self,token) :
        if self.currentToken != token :
            self.error(token + ' expected')
        else :
            self.scan()
    
    def parse(self) :
        self.scanner.init()
        self.scan()
        self.start()
        
    def Stmt(self) :
        self.Assign()
        self.expect('semicolon')
    
    def Assign(self) :
        self.LVal()
        self.expect('assignSym')
        self.AExpr()
        self.emit('op','STORE')
        
    def LVal(self) :
        if self.currentToken == 'identifier' :
            self.emit('lValue',self.currentValue)
            self.scan()

    def AExpr(self) :
        self.Term()
        while self.currentToken in {'plusSym', 'minusSym',} :
            op = self.currentToken
            self.scan()
            self.Term()
            self.emit('op',op)

    def Term(self) :
        self.Factor()
        while self.currentToken in {'timesSym', 'divSym',} :
            op = self.currentToken
            self.scan()
            self.Factor()
            self.emit('op',op)
            
    def Factor(self) :
        if self.currentToken == 'lParen' :
            self.scan()
            self.AExpr()
            self.expect('rParen');
                
        if self.currentToken == 'identifier' :
            self.emit('rValue',self.currentValue)
            self.scan()
        
        if self.currentToken == 'numConstant' :
            self.emit('numC',self.currentValue)
            self.scan() 

In [33]:
s = "xyz = count + 5*(a + 4) - 6;"
p = Parser(s)
p.parse()

['lValue', 'xyz']
['rValue', 'count']
['numC', 5]
['rValue', 'a']
['numC', 4]
['op', 'plusSym']
['op', 'timesSym']
['op', 'plusSym']
['numC', 6]
['op', 'minusSym']
['op', 'STORE']
